# 2A

In [70]:
import numpy as np

def segmented_least_squares(data):
    # Sort data based on x-coordinate
    data.sort(key=lambda x: x[0])
    n = len(data)

    # Compute Li,j for all valid i and j
    def compute_Li_j(i, j):
        x_sum = np.sum([data[k][0] for k in range(i, j + 1)])
        y_sum = np.sum([data[k][1] for k in range(i, j + 1)])
        xy_sum = np.sum([data[k][0] * data[k][1] for k in range(i, j + 1)])
        x_squared_sum = np.sum([data[k][0] ** 2 for k in range(i, j + 1)])

        bi_j = ((j - i + 1) * xy_sum - x_sum * y_sum) / ((j - i + 1) * x_squared_sum - x_sum ** 2)
        ai_j = (y_sum - bi_j * x_sum) / (j - i + 1)

        squared_error = (j - i + 1) * ai_j ** 2 + 2 * ai_j * bi_j * x_sum + bi_j ** 2 * x_squared_sum \
                        - 2 * ai_j * y_sum - 2 * bi_j * xy_sum + np.sum([data[k][1] ** 2 for k in range(i, j + 1)])

        return squared_error, ai_j, bi_j

    # Dynamic Programming (DP) function
    def dp(j, k):
        # Check if the result for this subproblem is already memoized
        if dp_array[j][k] is not None:
            return dp_array[j][k]

        # Base case: when k is 1, compute coefficients for the first line segment
        if k == 1:
            squared_error, ai_j, bi_j = compute_Li_j(0, j)
            dp_array[j][k] = squared_error, [(ai_j, bi_j, 0, j)]  # Store indices of the segment
            return squared_error, [(ai_j, bi_j, 0, j)]

        # Initialize variables to find the optimal partition
        min_error = float('inf')
        best_partition = None

        # Iterate through possible partition points
        for i in range(1, j):
            if 2 * k - 1 <= i <= j - 1:
                # Recursively compute the errors for the left and right segments
                squared_error1, coefficients1 = dp(i - 1, k - 1)
                squared_error2, ai_j, bi_j = compute_Li_j(i, j)
                
                # Combine errors and update the minimum error
                total_error = squared_error1 + squared_error2
                if total_error < min_error:
                    min_error = total_error
                    best_partition = coefficients1 + [(ai_j, bi_j, i, j)]

        # Memoize the result for this subproblem
        dp_array[j][k] = min_error, best_partition
        return min_error, best_partition

    # Initialize the 2D array with None values for memoization
    dp_array = [[None] * (num_segments+1) for _ in range(n)]

    # Perform dynamic programming to find optimal partitions
    _, partitions = dp(n - 1, num_segments)
    return partitions

# Load data from file
with open('seg_ls.txt', 'r') as file:
    n = int(file.readline().strip())
    data = [tuple(map(float, line.strip().split())) for line in file.readlines()]

# Apply segmented least squares algorithm
num_segments = 3
partitions = segmented_least_squares(data)

# Print results
for i, (a, b, start_idx, end_idx) in enumerate(partitions):
    x_start, y_start = data[start_idx]
    x_end, y_end = data[end_idx]

    print(f"Segment {i + 1}: y = {round(a, 4)} + {round(b, 4)}x, Endpoints: ({x_start}, {y_start}) to ({x_end}, {y_end})")


Segment 1: y = 41.417 + 2.6199x, Endpoints: (0.599, 8.2329) to (168.6967, 446.1948)
Segment 2: y = 499.5263 + -0.0828x, Endpoints: (170.4067, 447.1715) to (341.5237, 435.9841)
Segment 3: y = 1428.2028 + -2.7978x, Endpoints: (343.1398, 441.3405) to (498.5574, -0.9299)


# 2B

In [1]:
import numpy as np

def segmented_least_squares_arbitrary(data, cost_per_line=10000):
    data.sort(key=lambda x: x[0])  # Sort data based on x-coordinate
    n = len(data)

    # Initialize an array to store optimal solutions for subproblems
    dp_array = [None] * (n + 1)

    # Base case: no points
    dp_array[0] = (0, [])

    # Iterate through subproblems in a bottom-up manner
    for j in range(1, n + 1):
        min_error = float('inf')
        best_partition = None

        # Iterate through possible partition points
        for i in range(1, j + 1):
            # Calculate the error and coefficients for the current partition
            x_sum = np.sum([data[k][0] for k in range(i - 1, j)])
            y_sum = np.sum([data[k][1] for k in range(i - 1, j)])
            xy_sum = np.sum([data[k][0] * data[k][1] for k in range(i - 1, j)])
            x_squared_sum = np.sum([data[k][0] ** 2 for k in range(i - 1, j)])

            # Check for potential division by zero
            denominator = (j - i + 1) * x_squared_sum - x_sum ** 2
            if np.isclose(denominator, 0):
                # Handle division by zero or very small denominator
                bi_j = 0  # You can set it to a default value or handle it based on your specific case
            else:
                # Calculate coefficients for the linear equation in the current partition
                bi_j = ((j - i + 1) * xy_sum - x_sum * y_sum) / denominator
            ai_j = (y_sum - bi_j * x_sum) / (j - i + 1)

            # Directly calculate the error for the current partition
            squared_error = (j - i + 1) * ai_j ** 2 + 2 * ai_j * bi_j * x_sum + bi_j ** 2 * x_squared_sum \
                            - 2 * ai_j * y_sum - 2 * bi_j * xy_sum + np.sum([data[k][1] ** 2 for k in range(i - 1, j)])

            # Retrieve the solution for the remaining cost and coefficients from the table
            remaining_cost, coefficients = dp_array[i - 1]

            # Update the total error with the cost per line
            total_error = squared_error + remaining_cost + cost_per_line

            # Update minimum error and best partition
            if total_error < min_error:
                min_error = total_error
                best_partition = coefficients + [(ai_j, bi_j, i - 1, j - 1)]

        # Store the optimal solution for the current subproblem in the table
        dp_array[j] = min_error, best_partition

    # Retrieve the final solution for the original problem
    _, partitions = dp_array[n]
    return partitions

# Load data from file
with open('seg_ls.txt', 'r') as file:
    n = int(file.readline().strip())
    data = [tuple(map(float, line.strip().split())) for line in file.readlines()]

# Apply segmented least squares algorithm for an arbitrary number of lines
partitions = segmented_least_squares_arbitrary(data)

# Print results with endpoints
for i, (a, b, start_idx, end_idx) in enumerate(partitions):
    x_start, y_start = data[start_idx]
    x_end, y_end = data[end_idx]

    print(f"Segment {i + 1}: y = {round(a, 4)} + {round(b, 4)}x, Endpoints: ({x_start}, {y_start}) to ({x_end}, {y_end})")


Segment 1: y = 11.4751 + 3.2699x, Endpoints: (0.599, 8.2329) to (86.9667, 294.2807)
Segment 2: y = 125.4943 + 1.9756x, Endpoints: (88.7357, 287.2646) to (163.5283, 445.1173)
Segment 3: y = 334.124 + 0.7051x, Endpoints: (164.7958, 433.1884) to (250.3958, 499.4984)
Segment 4: y = 704.0777 + -0.7672x, Endpoints: (252.0358, 505.7027) to (341.5237, 435.9841)
Segment 5: y = 1182.4614 + -2.1598x, Endpoints: (343.1398, 441.3405) to (421.9327, 265.6377)
Segment 6: y = 1693.9691 + -3.3707x, Endpoints: (422.7515, 262.1774) to (498.5574, -0.9299)


# 3A

In [57]:
def canFitThreeContainers(weights, M):
    n = len(weights)
    
    # Initialize a 3D DP array to store intermediate results
    dp = [[[False] * (M+1) for _ in range(M+1)] for _ in range(n+1)]

    # Base case: All containers are empty
    dp[0][0][0] = True

    # Dynamic Programming: Fill the DP array
    for i in range(1, n+1):
        w = weights[i-1]
        for j in range(M+1):
            for k in range(M+1):
                # Case 1: Exclude the current bag from both containers
                dp[i][j][k] = dp[i-1][j][k]
                # Case 2: Include the current bag in the first container
                if j >= w:
                    dp[i][j][k] |= dp[i-1][j-w][k]
                # Case 3: Include the current bag in the second container
                if k >= w:
                    dp[i][j][k] |= dp[i-1][j][k-w]

    # Check if there is a way to distribute the bags into three containers
    for j in range(M+1):
        for k in range(M+1):
            # Weight of the third container
            third_container_weight = sum(weights) - j - k
            # Check if the weights satisfy the conditions and if DP value is True
            if max(j, k, third_container_weight) <= M and dp[n][j][k]:
                return True

    return False


#Examples
w = [2,10,5,3,6,1,6]  # Replace with your bag weights
M = 11  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print True

w = [2,3,5,6,6,10,2]  # Replace with your bag weights
M = 11  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print False

w = [3,3,3,3,3,3]  # Replace with your bag weights
M = 7  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print True

w = [3,3,3,3,3,3,3]  # Replace with your bag weights
M = 7  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print False

True
False
True
False


# 3B

In [64]:
def canFitThreeContainers(weights, M):
    n = len(weights)
    
    # Initialize a 3D DP array to store intermediate results
    dp = [[[False] * (M+1) for _ in range(M+1)] for _ in range(n+1)]

    # Base case: All containers are empty
    dp[0][0][0] = True

    # Dynamic Programming: Fill the DP array
    for i in range(1, n+1):
        w = weights[i-1]
        for j in range(M+1):
            for k in range(M+1):
                # Case 1: Exclude the current bag from both containers
                dp[i][j][k] = dp[i-1][j][k]
                # Case 2: Include the current bag in the first container
                if j >= w:
                    dp[i][j][k] |= dp[i-1][j-w][k]
                # Case 3: Include the current bag in the second container
                if k >= w:
                    dp[i][j][k] |= dp[i-1][j][k-w]

    # Check if there is a way to distribute the bags into three containers
    for j in range(M+1):
        for k in range(M+1):
            # Remaining
            remaining_weight = sum(weights) - j - k
            # Check if the weights satisfy the conditions and if DP value is True
            if max(j, k) <= M and dp[n][j][k] and remaining_weight==0:
                return True

    return False


#Examples
w = [2,10,3,1,6]  # Replace with your bag weights
M = 11  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print True

w = [2,3,10,6,1]  # Replace with your bag weights
M = 11  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print True

w = [3,3,3,3,3,3]  # Replace with your bag weights
M = 7  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print False

w = [3,3,3,3]  # Replace with your bag weights
M = 7  # Replace with container capacity
print(canFitThreeContainers(w, M)) # Should print True

True
True
False
True


# 4

In [16]:
def max_subarray_sum_with_constraints(A, X):
    n = len(A)
    
    # Initialize DP array to store the maximum subarray sum ending at each index
    dp = [0] * n
    
    # Initialize variables to keep track of the starting and ending indices of the subarray
    start_index = 0
    end_index = 0
    
    # Initialize the first element of the DP array
    dp[0] = A[0]
    
    # Iterate through the array to fill the DP array
    for i in range(1, n):
        dp[i] = max(A[i], dp[i-1] + A[i])
        
        # Update the starting index when a new subarray begins
        if dp[i] == A[i]:
            start_index = i
        
        # Update the maximum subarray sum and indices to be output only if subarray meets length constraint
        if dp[i] > dp[end_index] and i - start_index + 1 >= X:
            end_index = i
    
    return dp[end_index], start_index, end_index

# Example usage:
A = [-2, 10, -11, 4, -1, 2, 1, -5, 4]
X = 3
max_sum, L, R = max_subarray_sum_with_constraints(A, X)

print("Maximum Subarray Sum:", max_sum)
print("Start Index:", L)
print("End Index:", R)


Maximum Subarray Sum: 6
Start Index: 3
End Index: 6


# 5

In [8]:
def count_ways_to_shuffle(A, B, C):
    m, n, p = len(A), len(B), len(C)

    # Initialize the 2D array dp
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Base cases
    dp[0][0] = 1

    for i in range(1, m + 1):
        dp[i][0] = dp[i-1][0] if A[i-1] == C[i-1] else 0

    for j in range(1, n + 1):
        dp[0][j] = dp[0][j-1] if B[j-1] == C[j-1] else 0

    # Fill in the dp array using the recurrence relation
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if A[i-1] == C[i+j-1]:
                dp[i][j] += dp[i-1][j]
            if B[j-1] == C[i+j-1]:
                dp[i][j] += dp[i][j-1]

    return dp[m][n]

# Example usage
A = "BANANA"
B = "ANANAS"
C = "BANANAANANAS"
result = count_ways_to_shuffle(A, B, C)
print("Number of ways that C is a shuffle of A and B:", result)




Number of ways that C is a shuffle of A and B: 12
